# Amazon Recommendation System

Build a simple recommendation system for an Amazon-like store that uses user ratings and item similarity to deliver personalized product suggestions (to improve UX and conversions).

[Amazon Recommendation System using Python](https://amanxai.com/2021/03/23/amazon-recommendation-system-using-python/)

<img src="https://uploads-ssl.webflow.com/60f03643ffba6a48a3bda298/6287299a8211a80d33096200_amazon%20banner.png" width="750">

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("amazon.csv")

In [3]:
data.head()

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [4]:
data.columns = ['user_id', 'product_id','ratings','timestamp']

In [5]:
data.shape

(7824481, 4)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824481 entries, 0 to 7824480
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     object 
 1   product_id  object 
 2   ratings     float64
 3   timestamp   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 238.8+ MB


In [7]:
data.isnull().sum()

user_id       0
product_id    0
ratings       0
timestamp     0
dtype: int64

In [8]:
df = data[:int(len(data) * .1)] # Veri büyük olduğu için  datanın  ilk  %10 nunu alıyoruz 

In [9]:
data.shape

(7824481, 4)

In [10]:
counts = df['user_id'].value_counts()
# Kullanıcının kaç değerlendirme  yaptığına bakıyoruz .

In [11]:
data = df[df['user_id'].isin(counts[counts >= 50].index)]
# En az 50 değerlendirme yapan kullanıcıların verilerini filtreler.

In [12]:
data.groupby('product_id')['ratings'].mean().sort_values(ascending=False)
# Her ürün için ortalama puanı hesaplıyoruz ve yüksekten düşüğe sıralıyoruz 

product_id
B000068IGI    5.0
B00006IJO4    5.0
B00006JI6T    5.0
B0001CNKCM    5.0
B000816XI4    5.0
             ... 
B00005QWZ7    1.0
B00009W5QR    1.0
B00005RKO5    1.0
B00005UV2R    1.0
1400532655    1.0
Name: ratings, Length: 2207, dtype: float64

In [13]:
final_ratings = data.pivot(index='user_id', columns='product_id', values='ratings').fillna(0)
# Satırlarda kullanıcılar, sütunlarda ürünler olacak şekilde kullanıcı-ürün puan matrisi oluşturur.
# Eksik puanları 0 ile doldurur (matrisi tam yapmak için).

In [14]:
num_of_ratings = np.count_nonzero(final_ratings)
# final_ratings matrisinde kaç tane sıfır olmayan puan (yani gerçek değerlendirme) olduğunu sayar.

In [15]:
possible_ratings = final_ratings.shape[0] * final_ratings.shape[1]
# Matrisin toplam hücre sayısını bulur (kullanıcı sayısı × ürün sayısı = olası tüm değerlendirmeler).

In [16]:
density = (num_of_ratings / possible_ratings)
# Gerçek yapılan değerlendirmelerin oranını hesaplar.

In [17]:
density *= 100 #ORanı % cinsine cevirdik

In [18]:
final_ratings_T = final_ratings.transpose()
# Matrisi ters çevirirdik . Böylece ürünleri satır , kullanıcılıarı sütun yaptık

In [19]:
grouped = data.groupby('product_id').agg({'user_id': 'count'}).reset_index()
# Her ürün için kaç farklı kullancı var onu bulduk 

In [20]:
grouped.rename(columns={'user_id': 'score'}, inplace=True)
# Bu sayım sütunun adını score yaptık

In [21]:
training_data = grouped.sort_values(['score', 'product_id'], ascending=[0, 1])
# Ürünleri önce 'score' yüksekten düşüğe, eşitlikte 'product_id' artan sırada sıralar.

In [22]:
training_data['Rank'] = training_data['score'].rank(ascending=0, method='first')
# Skoru büyük olan ürüne daha iyi (küçük) sıra numarası verir; eşitlikte ilk gelen önce.

In [23]:
recommendations = training_data.head()

In [24]:
def recommend(id):     
    recommend_products = recommendations # hazırlanmış tabloyu referans aldık
    recommend_products['user_id'] = id    # Sonuc tablsosunda önerilerin kime ait old yazdık 
    column = recommend_products.columns.tolist()  # sütun adlarını listeye cevirdik
    column = column[-1:] + column[:-1]       # son sütun öne alındı
    recommend_products = recommend_products[column]   #sütunları yeni sıraya göre diziyoruz. 
    return recommend_products 

In [25]:
print(recommend(11))

      user_id  product_id  score  Rank
113        11  B00004SB92      6   1.0
1099       11  B00008OE6I      5   2.0
368        11  B00005AW1H      4   3.0
612        11  B0000645C9      4   4.0
976        11  B00007KDVI      4   5.0


C:\Users\asus\AppData\Local\Temp\ipykernel_4028\689612046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_products['user_id'] = id    # Sonuc tablsosunda önerilerin kime ait old yazdık


## Conclusiom
This project successfully built a simple Amazon-like recommendation system using Python, demonstrating how user ratings and item similarities can be used to generate personalized product suggestions.